In [1]:
from random import randint
import sys
import json
import requests
from requests.exceptions import ConnectTimeout, SSLError, ReadTimeout, ConnectionError
import numpy as np
import pandas as pd
from GolemQ.fetch.crawler import (
    headers,
)
from GolemQ.utils.parameter import (
    AKA, 
    INDICATOR_FIELD as FLD, 
    TREND_STATUS as ST,
    FEATURES as FTR,
)
import QUANTAXIS as QA
from QUANTAXIS.QAUtil import (
        DATABASE,
        QA_util_code_tolist,
    QA_util_date_stamp,
    QA_util_date_valid,
    QA_util_log_info,
)
from QUANTAXIS.QAUtil import (
        QA_util_to_json_from_pandas
)
from QUANTAXIS.QAUtil.QADate_trade import (
        QA_util_get_pre_trade_date,
        QA_util_if_tradetime,
)
import time
from datetime import (
    datetime as dt,
    timezone, timedelta
)
import datetime
import pymongo
import traceback
from tqdm.notebook import trange, tqdm


def GQ_fetch_stock_concept_components(concept='BK0066',
                                      delay_gap=0.8,
                                      pagenumber=1,
                                      pagelimit=400,
                                      concept_name=None,):
    '''
    抓取东方财富板块概念成分数据
    http://28.push2.eastmoney.com/api/qt/clist/get?cb=jQuery112405892331704393758_1629080255227&pn=2&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=b:BK0981+f:!50&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152,f45&_=1629080255234    
    '''
    def get_random_stock_concept_components_url():
        url = "http://{:2d}.push2.eastmoney.com/api/qt/clist/get".format(randint(1, 99))
        return url
    
    params = {
        "pn": pagenumber,
        "pz": pagelimit,
        "po": '1',
        "np": '1',
        "fltt": '2',
        "invt": '2',
        'fid': 'f3',
        'fs': 'b:{:s}+f:!50'.format(concept),
        "fields": "f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152,f45",
        "ut": "bd1d9ddb04089700cf9c27f6f7426281",
        "cb": "jQuery1124023986497915529914_{:d}".format(int(dt.utcnow().timestamp())),
        "_": int(time.time() * 1000),
    }
    
    retries = 1
    while (retries != 0):
        try:
            url = get_random_stock_concept_components_url()
            r = requests.get(url, params=params, headers=headers)
            retries = 0
        except (ConnectTimeout, ConnectionError, SSLError, ReadTimeout):
            retries = retries + 1
            if (retries % 18 == 0):
                print("Retry {} #{}".format(retries - 1))
            time.sleep(delay_gap)
    text_data = r.text
    json_data = json.loads(text_data[text_data.find("{") : -2])
    #print(concept, json_data)
    if (isinstance(json_data, dict)):
        if (json_data["data"] is None):
            return None
        try:
            content_list = json_data["data"]["diff"]
        except Exception as e:
            print(type(json_data))
            print(json_data)
            traceback.print_exception(type(e), e, sys.exc_info()[2])
            print(u'Failed:{}\n'.format(stock), e) 
            return None
        #print(json_data)

        temp_df = pd.DataFrame([item for item in content_list])
        #print(temp_df)
        ret_stock_concept_components = temp_df.rename(columns={'f2':AKA.CLOSE, 
                                          'f3':FLD.PCT_CHANGE, 
                                          'f4':'delta', 
                                          'f12':AKA.CODE, 
                                          'f14':AKA.NAME})
        ret_stock_concept_components['concept_symbol'] = concept
        ret_stock_concept_components['concept_name'] = concept_name
        ret_stock_concept_components[FLD.PCT_CHANGE] = np.where(ret_stock_concept_components[FLD.PCT_CHANGE] == '-', 
                                                        0.0, ret_stock_concept_components[FLD.PCT_CHANGE])
        ret_stock_concept_components['delta'] = np.where(ret_stock_concept_components['delta'] == '-', 
                                                        0.0, ret_stock_concept_components['delta'])
        ret_stock_concept_components[AKA.CLOSE] = np.where(ret_stock_concept_components[AKA.CLOSE] == '-', 
                                                        0.0, ret_stock_concept_components[AKA.CLOSE])
        ret_stock_concept_components[FLD.PCT_CHANGE] = ret_stock_concept_components[FLD.PCT_CHANGE] / 100
        ret_stock_concept_components[FLD.SOURCE] = 'eastmoney'
        #print( ret_stock_concept_components)
        return  ret_stock_concept_components
    
    return 


def fetch_stock_concept_list_from_eastmoney(delay_gap=0.8,
                                       pagenumber=1,
                                       pagelimit=400,
                                       model='concept'):
    '''
    抓取东方财富板块概念数据
    http://76.push2.eastmoney.com/api/qt/clist/get?cb=jQuery1124023986497915529914_1628836522452&pn=1&pz=20&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:90+t:3+f:!50&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f26,f22,f33,f11,f62,f128,f136,f115,f152,f124,f107,f104,f105,f140,f141,f207,f208,f209,f222&_=1628836522455
    http://58.push2.eastmoney.com/api/qt/clist/get?cb=jQuery112406314256519339916_1629966378474&pn=1&pz=400&po=1&np=1&ut=bd1d9ddb04089700cf9c27f6f7426281&fltt=2&invt=2&fid=f3&fs=m:90+t:2+f:!50&fields=f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f26,f22,f33,f11,f62,f128,f136,f115,f152,f124,f107,f104,f105,f140,f141,f207,f208,f209,f222&_=1629966378479
    '''
    def get_random_stock_concept_url():
        url = "http://{:2d}.push2.eastmoney.com/api/qt/clist/get".format(randint(1, 99))
        return url
    
    params = {
        "pn": pagenumber,
        "pz": pagelimit,
        "np": '1',
        "fltt": '2',
        "invt": '2',
        'fid': 'f3',
        'fs': 'm:90+t:3+f:!50' if (model=='concept') else 'm:90+t:2+f:!50',
        "fields": "f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f26,f22,f33,f11,f62,f128,f136,f115,f152,f124,f107,f104,f105,f140,f141,f207,f208,f209,f222",
        "ut": "bd1d9ddb04089700cf9c27f6f7426281",
        "cb": "jQuery1124023986497915529914_{:d}".format(int(dt.utcnow().timestamp())),
        "_": int(time.time() * 1000),
    }
    
    retries = 1
    while (retries != 0):
        try:
            url = get_random_stock_concept_url()
            r = requests.get(url, params=params, headers=headers)
            retries = 0
        except (ConnectTimeout, ConnectionError, SSLError, ReadTimeout):
            retries = retries + 1
            if (retries % 18 == 0):
                print("Retry {} #{}".format(retries - 1))
            time.sleep(delay_gap)
    text_data = r.text
    json_data = json.loads(text_data[text_data.find("{") : -2])
    #print(stock_bk, json_data)
    if (isinstance(json_data, dict)):
        if (json_data["data"] is None):
            return None
        try:
            content_list = json_data["data"]["diff"]
        except Exception as e:
            print(type(json_data))
            print(json_data)
            traceback.print_exception(type(e), e, sys.exc_info()[2])
            print(u'Failed:{}\n'.format(stock), e) 
            return None
        #print(json_data)

        temp_df = pd.DataFrame([item for item in content_list])
        #print(temp_df)
        ret_stock_concept_list = temp_df.rename(columns={'f2':AKA.CLOSE, 
                                          'f3':FLD.PCT_CHANGE, 
                                          'f4':'delta', 
                                          'f12':AKA.SYMBOL, 
                                          'f14':AKA.NAME})
        ret_stock_concept_list[FLD.PCT_CHANGE] = np.where(ret_stock_concept_list[FLD.PCT_CHANGE] == '-', 
                                                        0.0, ret_stock_concept_list[FLD.PCT_CHANGE])
        ret_stock_concept_list['delta'] = np.where(ret_stock_concept_list['delta'] == '-', 
                                                        0.0, ret_stock_concept_list['delta'])
        ret_stock_concept_list[FLD.PCT_CHANGE] = ret_stock_concept_list[FLD.PCT_CHANGE] / 100
        ret_stock_concept_list[FLD.SOURCE] = 'eastmoney'
        ret_stock_concept_list[FLD.MODEL] = 'concept' if (model=='concept') else 'industry'
        return ret_stock_concept_list
    

def GQ_save_stock_concept_kline(stock_concept_kline_df=None, 
                                freq=QA.FREQUENCE.HOUR,
    collections=DATABASE.stock_concept_min):
    """
    东方财富股票概念板块K线数据，将其保存在数据库
    """    
    if (freq==QA.FREQUENCE.DAY):
        collections = DATABASE.stock_concept_day
        coll = DATABASE.stock_concept_day
        coll.create_index([(AKA.SYMBOL,
                     pymongo.ASCENDING),
                    ("date_stamp",
                     pymongo.ASCENDING)],
                unique=True)
    else:
        collections = DATABASE.stock_concept_min
        coll = DATABASE.stock_concept_min
        coll.create_index([(AKA.SYMBOL,
                         pymongo.ASCENDING),
                        ("type",
                         pymongo.ASCENDING),
                        (FLD.DATETIME,
                         pymongo.ASCENDING),],
                    unique=False)
        coll.create_index([(AKA.SYMBOL,
                        pymongo.ASCENDING),
                    ("type",
                        pymongo.ASCENDING),
                    ("time_stamp",
                        pymongo.ASCENDING),],
                unique=True)
        coll.create_index([(AKA.SYMBOL,
                        pymongo.ASCENDING),
                    ("type",
                        pymongo.ASCENDING),
                    ("date_stamp",
                        pymongo.ASCENDING),],
                unique=False)

    data = stock_concept_kline_df

    # 查询是否新数据
    if (freq==QA.FREQUENCE.DAY):
        query_id = {
                        AKA.SYMBOL: data.iloc[0].symbol,
                        'date_stamp': {
                            '$in': data['date_stamp'].tolist()
                        }
                    }
    else:
        query_id = {
                        AKA.SYMBOL: data.iloc[0].symbol,
                        'type': freq,
                        'time_stamp': {
                            '$in': data['time_stamp'].tolist()
                        }
                    }
    refcount = coll.count_documents(query_id)
    
    try:
        if refcount > 0:
             if (len(data) > 1):
                 # 删掉重复数据
                 coll.delete_many(query_id)
                 data = QA_util_to_json_from_pandas(data)
                 coll.insert_many(data)
             else:
                 # 持续接收行情，更新记录
                 if ('created_at' in data.columns):
                     data.drop('created_at', axis=1, inplace=True)
                 data = QA_util_to_json_from_pandas(data)
                 coll.replace_one(query_id, data[0])
        else:
             # 新 tick，插入记录
             data = QA_util_to_json_from_pandas(data)
             coll.insert_many(data)

    except Exception as e:
        if (data is not None):
            traceback.print_exception(type(e), e, sys.exc_info()[2])
            print(u'Failed:{}\n'.format(code), e) 
                        
    return 


def GQ_save_stock_concept_components(stock_concept_components_df=None,
    collections=DATABASE.stock_concept_components):
    """
    获取东方财富股票概念板块成分数据，并将其保存在数据库
    """
    coll = collections
    coll.create_index([('concept_symbol',pymongo.ASCENDING),(AKA.CODE,pymongo.ASCENDING),("revision",pymongo.ASCENDING)],
            unique=True)
    coll.create_index([('concept_symbol',pymongo.ASCENDING),(AKA.NAME,pymongo.ASCENDING),("revision",pymongo.ASCENDING)],unique=True)
    coll.create_index([(AKA.CODE,pymongo.ASCENDING),("revision",pymongo.ASCENDING)],unique=False)
    
    column_list = [AKA.CLOSE, FLD.PCT_CHANGE, 'delta', ]
    stock_concept_components_df.loc[:, column_list] = stock_concept_components_df[column_list].astype(np.float64)
    print(stock_concept_components_df)
    
    date_epoch = dt.now().date()
    if not QA_util_if_tradetime(dt.now()):
        date_epoch = QA_util_get_pre_trade_date('{}'.format(datetime.date.today()), n=0)

    stock_concept_components_df['date'] = pd.to_datetime(date_epoch)
    stock_concept_components_df['revision'] = pd.to_datetime(stock_concept_components_df['date']).astype(np.int64) // 10 ** 9
    stock_concept_components_df['date_stamp'] = pd.to_datetime(stock_concept_components_df['date']).astype(np.int64) // 10 ** 9

    data = stock_concept_components_df
    #print((each_day[-1], slice(None)))
    # 查询是否新数据
    query_id = {
        'concept_symbol':data['concept_symbol'].tail(1).item(),
        AKA.CODE: {
                 '$in': data[AKA.CODE].tolist()
                },
        "revision": int(data.tail(1).date_stamp),
    }
    refcount = coll.count_documents(query_id)

    if refcount > 0:
            if (len(data) > 1):
                # 删掉重复数据
                coll.delete_many(query_id)

                # 作为差量更新，只更新最后一天的数据
                data = QA_util_to_json_from_pandas(data)
                coll.insert_many(data)
            else:
                # 持续接收行情，更新记录
                if ('created_at' in data.columns):
                    data.drop('created_at', axis=1, inplace=True)
                data = QA_util_to_json_from_pandas(data)
                coll.replace_one(query_id, data[0])
    else:
        # 新 tick，插入记录
        try:
            data = QA_util_to_json_from_pandas(data)
            coll.insert_many(data)
        except Exception as e:
            traceback.print_exception(type(e), e, sys.exc_info()[2])
            print(e)
            return None
        
    return stock_concept_components_df


def GQ_SU_crawl_stock_concept_from_eastmoney(
    collections=DATABASE.stock_concept_list,
    model='concept'):
    """
    获取东方财富股票概念板块数据，并将其保存在数据库
    """
    stock_concept_list_df = fetch_stock_concept_list_from_eastmoney(model=model)[[AKA.CLOSE, 
                                                                           FLD.PCT_CHANGE, 
                                                                           'delta', 
                                                                           AKA.SYMBOL, 
                                                                           AKA.NAME, 
                                                                           FLD.SOURCE]].copy()
        
    coll = collections
    coll.create_index([(AKA.SYMBOL,
                 pymongo.ASCENDING),
                ("revision",
                 pymongo.ASCENDING)],
            unique=True)
    
    column_list = [AKA.CLOSE, FLD.PCT_CHANGE, 'delta', ]
    stock_concept_list_df.loc[:, column_list] = stock_concept_list_df[column_list].astype(np.float64)
    
    date_epoch = dt.now().date()
    if not QA_util_if_tradetime(dt.now()):
        date_epoch = QA_util_get_pre_trade_date('{}'.format(datetime.date.today()), n=0)

    stock_concept_list_df['date'] = pd.to_datetime(date_epoch)
    stock_concept_list_df['revision'] = pd.to_datetime(stock_concept_list_df['date']).astype(np.int64) // 10 ** 9
    stock_concept_list_df['date_stamp'] = pd.to_datetime(stock_concept_list_df['date']).astype(np.int64) // 10 ** 9

    data = stock_concept_list_df
    #print((each_day[-1], slice(None)))
    # 查询是否新数据
    query_id = {
        AKA.SYMBOL: {
                 '$in': data[AKA.SYMBOL].tolist()
                },
        "revision": int(data.tail(1).date_stamp),
    }
    refcount = coll.count_documents(query_id)

    if refcount > 0:
            if (len(data) > 1):
                # 删掉重复数据
                coll.delete_many(query_id)

                # 作为差量更新，只更新最后一天的数据
                data = QA_util_to_json_from_pandas(data)
                coll.insert_many(data)
            else:
                # 持续接收行情，更新记录
                if ('created_at' in data.columns):
                    data.drop('created_at', axis=1, inplace=True)
                data = QA_util_to_json_from_pandas(data)
                coll.replace_one(query_id, data[0])
    else:
        # 新 tick，插入记录
        try:
            data = QA_util_to_json_from_pandas(data)
            coll.insert_many(data)
        except Exception as e:
            traceback.print_exception(type(e), e, sys.exc_info()[2])
            print(e)
            return None
        
    return stock_concept_list_df


def GQ_SU_crawl_stock_concept_components_from_eastmoney(concept="all",
                                         delay_gap=0.8,
                                         pagenumber=1,
                                         pagelimit=400,
    collections=DATABASE.stock_concept_components):
    """
    获取东方财富股票概念板块数据，并将其保存在数据库
    """
    stock_concept_list = GQ_SU_crawl_stock_concept_from_eastmoney(model='concept')
    stock_industry_list = GQ_SU_crawl_stock_concept_from_eastmoney(model='industry')
    stock_concept_list = pd.concat([stock_concept_list,
                                    stock_industry_list], axis=0)
    
    coll_components = collections

    def crawl_stock_concept_components(concept:str='BK05666', stock_concept_block:pd.DataFrame=None):
        '''
        抓取概念板块成分
        '''
        ret_stock_concept_components = GQ_fetch_stock_concept_components(concept=concept)
        ret_stock_concept_components[u'概念'] = stock_concept_block[AKA.NAME].item()
        ret_stock_concept_components[AKA.NAME] = stock_concept_block[AKA.NAME].item()
        last_tradedate = QA_util_get_pre_trade_date('{}'.format(datetime.date.today()), n=1)
        ret_stock_concept_components['date'] = last_tradedate
        ret_stock_concept_components['date'] = pd.to_datetime(ret_stock_concept_components['date'],)
        ret_stock_concept_components['date'] = ret_stock_concept_components['date'].dt.strftime('%Y-%m-%d')

        # GMT+0 String 转换为 UTC Timestamp
        ret_stock_concept_components["revision"] = pd.to_datetime(ret_stock_concept_components['date']).astype(np.int64) // 10 ** 9
        ret_stock_concept_components["date_stamp"] = pd.to_datetime(ret_stock_concept_components['date']).astype(np.int64) // 10 ** 9
        return ret_stock_concept_components
    
    def crawl_stock_concept_klines(concept:str='BK05666', stock_concept_block:pd.DataFrame=None):
        '''
        抓取概念板块行情
        '''
        return ret_stock_concept_klines
        
    def save_stock_concept_components(concept:str=None,
                                  data:pd.DataFrame=None,
                                  collections=collections):
        '''
        保存概念板块成分，保存概念板块行情。
        '''
        # 查询是否新数据
        GQ_save_stock_concept_components(data,)
        return

    if concept == "all":
        # 读取东方财富Choice数据的板块股票列表代码
        print(u"💪 一共需要获取 %d 个概念板块的成分股列表 , 需要大概 %d 分钟" % (len(stock_concept_list), 
                                                          (len(stock_concept_list) * delay_gap * 1.3) / 60))

        code_list = stock_concept_list[AKA.SYMBOL].to_list()
        code_metadata = {each_code:"fooo" for each_code in stock_concept_list[AKA.SYMBOL].to_list()}
        try:
            overall_progress = tqdm(code_list, unit='stock')
            for code in overall_progress:
                overall_progress.set_description(u"概念板块({})".format(code))
                overall_progress.update(1)
                try:
                    stock_concept_block = code
                    code_metadata = stock_concept_list.query('{}==\'{}\''.format(AKA.SYMBOL,code))
                    stock_concept_components_df = crawl_stock_concept_components(code, code_metadata)[[AKA.CLOSE, 
                                                                                                       FLD.PCT_CHANGE, 
                                                                                                       'delta', 
                                                                                                       AKA.CODE, 
                                                                                                       AKA.NAME, 
                                                                                                       'concept_symbol', 
                                                                                                       'concept_name', 
                                                                                                       FLD.SOURCE]].copy()

                    if (stock_concept_components_df is None):
                        continue
                        
                    data = stock_concept_components_df
                    save_stock_concept_components(code,
                                                  data,)
                    
                    ret_concept_kline_hour = fetch_stock_concept_from_eastmoney(stock_bk=code, 
                                   freq=QA.FREQUENCE.HOUR,)
                    
                    if (ret_concept_kline_hour is None):
                        continue
                    
                    GQ_save_stock_concept_kline(ret_concept_kline_hour, 
                                                freq=QA.FREQUENCE.HOUR)

                    ret_concept_kline_min = fetch_stock_concept_from_eastmoney(stock_bk=code, 
                                   freq=QA.FREQUENCE.FIFTEEN_MIN,)
                    
                    if (ret_concept_kline_min is None):
                        continue
                    
                    GQ_save_stock_concept_kline(ret_concept_kline_min, 
                                                freq=QA.FREQUENCE.FIFTEEN_MIN)
                    
                    ret_concept_kline_day = fetch_stock_concept_from_eastmoney(stock_bk=code, 
                                                                               freq=QA.FREQUENCE.DAY,)
                    
                    if (ret_concept_kline_day is None):
                        continue

                    GQ_save_stock_concept_kline(ret_concept_kline_day, 
                                                freq=QA.FREQUENCE.DAY)
                except Exception as e:
                    traceback.print_exception(type(e), e, sys.exc_info()[2])
                    print(u'Failed:{}\n'.format(code), e) 
                time.sleep(delay_gap)
        except KeyboardInterrupt:
            overall_progress.close()
            raise
        return
    else:
        # todo 检查股票代码是否合法
        # return
        #
        pass
        print(u'代码未完成')
        stock_concept_components_df = crawl_stock_concept_components(code, code_metadata)[[AKA.CLOSE, 
                                                                                           FLD.PCT_CHANGE, 
                                                                                           'delta', 
                                                                                           AKA.CODE, 
                                                                                           AKA.NAME, 
                                                                                           'concept_symbol', 
                                                                                           'concept_name', 
                                                                                           FLD.SOURCE]]
        data = stock_concept_components_df
        save_stock_concept_components(code, data,)
        return
    
    return stock_concept_component_df


def fetch_stock_concept_from_eastmoney(stock_bk: str="BK05666", 
                                        freq=QA.FREQUENCE.HOUR,) -> pd.DataFrame:
    """
    东方财富网 > 行情中心 > 沪深板块
    http://quote.eastmoney.com/center/hsbk.html
    :param stock: 股票代码
    :type stock: str
    :param market: 股票市场; 上海证券交易所: sh, 深证证券交易所: sz
    :type market: str
    :return: 近期个股的资金流数据
    :rtype: pandas.DataFrame
    http://push2his.eastmoney.com/api/qt/stock/kline/get?cb=jQuery112404939798105940868_1629173273789&secid=90.BK0917&ut=fa5fd1943c7b386f172d6893dbfba10b&fields1=f1%2Cf2%2Cf3%2Cf4%2Cf5&fields2=f51%2Cf52%2Cf53%2Cf54%2Cf55%2Cf56%2Cf57%2Cf58%2Cf59%2Cf60%2Cf61&klt=60&fqt=0&beg=19900101&end=20220101&_=1629173274401
    """
    url = "http://push2his.eastmoney.com/api/qt/stock/kline/get"
    params = {
        "fqt": "0",
        "klt": "101" if freq == QA.FREQUENCE.DAY else ("15" if freq == QA.FREQUENCE.FIFTEEN_MIN else "60"),
        "secid": f"90.{stock_bk}",
        "fields1": "f1,f2,f3,f4,f5",
        "fields2": "f51,f52,f53,f54,f55,f56,f57,f58",
        "ut": "fa5fd1943c7b386f172d6893dbfba10b",
        "cb": "jQuery1124005797095004732822_{:d}".format(int(dt.utcnow().timestamp())),
        "_": int(time.time() * 1000),
        'beg':'19900101',
        'end': '20250101',
    }
    r = requests.get(url, params=params, headers=headers)
    text_data = r.text
    json_data = json.loads(text_data[text_data.find("{") : -2])
    #print(stock_bk, json_data)
    if (isinstance(json_data, dict)):
        if (json_data["data"] is None):
            return None
        try:
            content_list = json_data["data"]["klines"]
        except Exception as e:
            print(type(json_data))
            print(json_data)
            traceback.print_exception(type(e), e, sys.exc_info()[2])
            print(u'Failed:{}\n'.format(stock_bk), e) 
            return None
        #print(json_data)

        temp_df = pd.DataFrame([item.split(",") for item in content_list])
        #print(temp_df)
        temp_df.columns = [AKA.DATETIME,
            AKA.OPEN,
            AKA.CLOSE,
            AKA.HIGH,
            AKA.LOW,
            AKA.VOLUME,
            AKA.AMOUNT,
            AKA.TURNOVER,]
        #temp_df = temp_df.iloc[:, :-2]
        #print(temp_df)
        temp_df.loc[:, [AKA.OPEN,
            AKA.CLOSE,
            AKA.HIGH,
            AKA.LOW,
            AKA.VOLUME,
            AKA.AMOUNT,
            AKA.TURNOVER,]] = temp_df[[AKA.OPEN,
            AKA.CLOSE,
            AKA.HIGH,
            AKA.LOW,
            AKA.VOLUME,
            AKA.AMOUNT,
            AKA.TURNOVER,]].astype(np.float64)
        temp_df[AKA.SYMBOL] = stock_bk
        temp_df[AKA.TURNOVER] = temp_df[AKA.TURNOVER] / 100

        if (freq == QA.FREQUENCE.HOUR) or \
            (freq == QA.FREQUENCE.FIFTEEN_MIN):
            if (freq == QA.FREQUENCE.HOUR):
                temp_df['type'] = QA.FREQUENCE.HOUR
            if (freq == QA.FREQUENCE.FIFTEEN_MIN):
                temp_df['type'] = QA.FREQUENCE.FIFTEEN_MIN

            temp_df['date'] = pd.to_datetime(temp_df[AKA.DATETIME])
            temp_df['date'] = temp_df['date'].dt.strftime('%Y-%m-%d')
            temp_df['datetime'] = pd.to_datetime(temp_df[AKA.DATETIME],)
            temp_df['datetime'] = temp_df['datetime'].dt.strftime('%Y-%m-%d %H:%M:%S')
            # GMT+0 String 转换为 UTC Timestamp
            temp_df['time_stamp'] = pd.to_datetime(temp_df['datetime']).astype(np.int64) // 10 ** 9
        elif (freq == QA.FREQUENCE.DAY):
            temp_df['date'] = pd.to_datetime(temp_df[AKA.DATETIME])
            temp_df['date'] = temp_df['date'].dt.strftime('%Y-%m-%d')

        temp_df["date_stamp"] = pd.to_datetime(temp_df['date']).astype(np.int64) // 10 ** 9

        #print(temp_df)
        return temp_df
    else:
        return None

提示：当前环境 pandas 版本高于 0.25，get_price 与 get_fundamentals_continuously 接口 panel 参数将固定为 False
注意：0.25 以上版本 pandas 不支持 panel，如使用该数据结构和相关函数请注意修改


In [2]:
def get_stock_concept_list(
    code=None,
    symbol=None,
    format='pd',
    collections=DATABASE.stock_concept_list,
):
    """
    获取概念板块列表
    """
    if code is not None:
        symbol = code
    if symbol is not None:
        symbol = QA_util_code_tolist(symbol)
        data = pd.DataFrame(
            [
                item for item in
                collections.find({AKA.SYMBOL: {
                    '$in': symbol
                }, 'revision':{ 
                    '$gt': QA_util_date_stamp(dt.now() - timedelta(days=10))
                }},
                                 batch_size=10000)
            ]
        ).drop(['_id'],
               axis=1)
        return data.set_index(AKA.SYMBOL, drop=False).drop_duplicates(AKA.SYMBOL, 'last')
    else:
        data = pd.DataFrame([item for item in collections.find()]
                           ).drop(['_id'],
                                  axis=1)
        return data.set_index(AKA.SYMBOL, drop=False).drop_duplicates(AKA.SYMBOL, 'last')
    
    
def get_stock_concepts(symbol=None,
    format='pd',
    collections=DATABASE.stock_concept_components):
    """
    获取个股涉及的概念板列表
    """
    if symbol is not None:
        symbol = QA_util_code_tolist(symbol)
        data = pd.DataFrame(
            [
                item for item in
                collections.find({AKA.CODE: {
                    '$in': symbol
                }, 'revision':{ 
                    '$gt': QA_util_date_stamp(dt.now() - timedelta(days=10))
                }},
                                 batch_size=10000)
            ]
        ).drop(['_id'],
               axis=1)
        return data.set_index('concept_symbol', drop=False).drop_duplicates(['concept_symbol', AKA.CODE], 'last')
    else:
        data = pd.DataFrame([item for item in collections.find()]
                           ).drop(['_id'],
                                  axis=1)
        return data.set_index('concept_symbol', drop=False).drop_duplicates(['concept_symbol', AKA.CODE], 'last')
    
    return


def get_stock_concept_components(symbol=None,
    format='pd',
    collections=DATABASE.stock_concept_components):
    """
    获取概念板块成分股列表
    """
    if symbol is not None:
        symbol = QA_util_code_tolist(symbol)
        data = pd.DataFrame(
            [
                item for item in
                collections.find({'concept_symbol': {
                    '$in': symbol
                }, 'revision':{ 
                    '$gt': QA_util_date_stamp(dt.now() - timedelta(days=10))
                }},
                                 batch_size=10000)
            ]
        ).drop(['_id'],
               axis=1)
        return data.set_index('concept_symbol', drop=False).drop_duplicates(['concept_symbol', AKA.CODE], 'last')
    else:
        data = pd.DataFrame([item for item in collections.find()]
                           ).drop(['_id'],
                                  axis=1)
        return data.set_index('concept_symbol', drop=False).drop_duplicates(['concept_symbol', AKA.CODE], 'last')
    
    return


def get_stock_concept_kline(symbol:str=None,
                            freq=QA.FREQUENCE.DAY,
    start:str=None,
    end:str=None,
    format:str='pd',
    collections=DATABASE.stock_concept_day) -> pd.DataFrame:
    """'获取股票资金流向'

    Returns:
        [type] -- [description]
    """
    if (freq!=QA.FREQUENCE.DAY):
        collections=DATABASE.stock_concept_min
    
    start = '{}'.format(datetime.date.today() - timedelta(days=2500)) if (start is None) else str(start)[0:10]
    end = '{}'.format(datetime.date.today() + timedelta(days=1)) if (end is None) else str(end)[0:10]
    #code= [code] if isinstance(code,str) else code

    # code checking
    symbol = QA_util_code_tolist(symbol)
    if QA_util_date_valid(end):
        cursor = collections.find({
                AKA.SYMBOL: {
                    '$in': symbol
                },
                "date_stamp":
                    {
                        "$lte": QA_util_date_stamp(end),
                        "$gte": QA_util_date_stamp(start)
                    }
            },
            {"_id": 0},
            batch_size=10000)
        #res=[QA_util_dict_remove_key(data, '_id') for data in cursor]

        res = pd.DataFrame([item for item in cursor])
        try:
            res = res.assign(date=pd.to_datetime(res.date)).drop_duplicates((['date',
                                'symbol'])).set_index(['date',
                                'symbol'],
                                    drop=False)
        except Exception as e:
            print(u'get_stock_concept_kline:{}'.format(symbol), e)
            print(res.columns)
            pass
            
        if format in ['P', 'p', 'pandas', 'pd']:
            return res
        elif format in ['json', 'dict']:
            return QA_util_to_json_from_pandas(res)
        # 多种数据格式
        elif format in ['n', 'N', 'numpy']:
            return numpy.asarray(res)
        elif format in ['list', 'l', 'L']:
            return numpy.asarray(res).tolist()
        else:
            print("QA Error get_stock_concept_kline format parameter %s is none of  \"P, p, pandas, pd , json, dict , n, N, numpy, list, l, L, !\" " % format)
            return None
    else:
        QA_util_log_info('QA Error get_stock_concept_kline data parameter start=%s end=%s is not right' % (start,
               end))
    return res

In [3]:
stock_concept_list = GQ_SU_crawl_stock_concept_from_eastmoney()
stock_concept_list.query("name==\'海绵城市\'")[[AKA.CLOSE, 
                                            FLD.PCT_CHANGE, 
                                            AKA.DELTA, 
                                            AKA.SYMBOL, 
                                            AKA.NAME, 
                                            FLD.SOURCE]]

,close,PCT_CHG,delta,symbol,name,source
183,753.99,0.0126,9.39,BK0724,海绵城市,eastmoney


In [4]:
stock_concept_list.query("name==\'盐湖提锂\'")[[AKA.CLOSE, FLD.PCT_CHANGE, 'delta', AKA.SYMBOL, AKA.NAME, FLD.SOURCE]]

,close,PCT_CHG,delta,symbol,name,source
310,1647.35,0.0279,44.77,BK0987,盐湖提锂,eastmoney


In [5]:
concept = stock_concept_list[AKA.SYMBOL].tail(2).head(1).item()
concept_name = stock_concept_list[AKA.NAME].tail(2).head(1).item()

concept = 'BK0708'
concept_name = u'体育产业'

stock_concept_components = GQ_fetch_stock_concept_components(concept=concept, concept_name=concept_name)[[AKA.CLOSE, FLD.PCT_CHANGE, 'delta', AKA.CODE, AKA.NAME, 'concept_symbol', 'concept_name', FLD.SOURCE]].copy()

GQ_save_stock_concept_components(stock_concept_components,)
stock_concept_components


    close  PCT_CHG  delta    code  name concept_symbol concept_name     source
0   22.95   0.1002   2.09  002346  柘中股份         BK0708         体育产业  eastmoney
1   23.55   0.1000   2.14  605299  舒华体育         BK0708         体育产业  eastmoney
2    4.04   0.0919   0.34  600052  浙江广厦         BK0708         体育产业  eastmoney
3   12.92   0.0749   0.90  603081  大丰实业         BK0708         体育产业  eastmoney
4   44.60   0.0529   2.24  603908   牧高笛         BK0708         体育产业  eastmoney
..    ...      ...    ...     ...   ...            ...          ...        ...
61   9.87  -0.0199  -0.20  300162  雷曼光电         BK0708         体育产业  eastmoney
62  11.44  -0.0214  -0.25  000811  冰轮环境         BK0708         体育产业  eastmoney
63   4.17  -0.0302  -0.13  600358  国旅联合         BK0708         体育产业  eastmoney
64  26.17  -0.0322  -0.87  002768  国恩股份         BK0708         体育产业  eastmoney
65  26.95  -0.0507  -1.44  300526  中潜股份         BK0708         体育产业  eastmoney

[66 rows x 8 columns]


,close,PCT_CHG,delta,code,name,concept_symbol,concept_name,source,date,revision,date_stamp
0,22.95,0.1002,2.09,002346,柘中股份,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
1,23.55,0.1000,2.14,605299,舒华体育,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
2,4.04,0.0919,0.34,600052,浙江广厦,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
3,12.92,0.0749,0.90,603081,大丰实业,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
4,44.60,0.0529,2.24,603908,牧高笛,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
...,...,...,...,...,...,...,...,...,...,...,...
61,9.87,-0.0199,-0.20,300162,雷曼光电,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
62,11.44,-0.0214,-0.25,000811,冰轮环境,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
63,4.17,-0.0302,-0.13,600358,国旅联合,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
64,26.17,-0.0322,-0.87,002768,国恩股份,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400


In [6]:
concept_symbol = stock_concept_list.query("name==\'海绵城市\'")
print(concept_symbol)
ret_concept_kline_hour = fetch_stock_concept_from_eastmoney(stock_bk=concept_symbol[AKA.SYMBOL].item(), 
                                   freq=QA.FREQUENCE.HOUR,)
print(ret_concept_kline_hour)
GQ_save_stock_concept_kline(ret_concept_kline_hour, 
                            freq=QA.FREQUENCE.HOUR)

ret_concept_kline_day = fetch_stock_concept_from_eastmoney(stock_bk=concept_symbol[AKA.SYMBOL].item(), 
                                   freq=QA.FREQUENCE.DAY,)
print(ret_concept_kline_day)
GQ_save_stock_concept_kline(ret_concept_kline_day, 
                            freq=QA.FREQUENCE.DAY)

      close  PCT_CHG  delta  symbol  name     source                 date  \
183  753.99   0.0126   9.39  BK0724  海绵城市  eastmoney  2021-09-06 00:00:00   

       revision  date_stamp  
183  1630886400  1630886400  
                datetime    open   close    high     low     volume  \
0    2021-07-23 10:30:00  688.42  685.22  688.94   684.7  4800959.0   
1    2021-07-23 11:30:00  685.05  684.47  686.67  683.81  1680494.0   
2    2021-07-23 14:00:00  684.49  686.55  687.49  682.88  1557813.0   
3    2021-07-23 15:00:00  686.49  685.51  686.89  684.55  1673615.0   
4    2021-07-26 10:30:00  685.36  679.67  685.36  676.97  3312810.0   
..                   ...     ...     ...     ...     ...        ...   
123  2021-09-03 15:00:00  746.35   744.6  746.49  743.54  3125682.0   
124  2021-09-06 10:30:00  745.35  745.19  751.42   742.8  5561691.0   
125  2021-09-06 11:30:00  745.31   750.3  750.85  744.76  1608842.0   
126  2021-09-06 14:00:00  750.33   751.0  752.01  748.74  1491085.0   
127 

In [7]:
#stock_concept_components = GQ_SU_crawl_stock_concept_components_from_eastmoney()

In [8]:
get_stock_concept_list()

,close,PCT_CHG,delta,symbol,name,source,date,revision,date_stamp
symbol,,,,,,,,,
BK0479,9537.61,-0.0047,-45.10,BK0479,钢铁行业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0729,718.08,-0.0036,-2.56,BK0729,船舶制造,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0476,14129.84,-0.0033,-47.18,BK0476,木业家具,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0454,11784.21,-0.0025,-30.04,BK0454,塑胶制品,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0478,15791.55,-0.0017,-26.52,BK0478,有色金属,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
...,...,...,...,...,...,...,...,...,...
BK0938,962.83,0.0440,40.58,BK0938,代糖概念,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0818,1103.71,0.0467,49.22,BK0818,可燃冰,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK1000,877.67,0.0536,44.63,BK1000,宁组合,eastmoney,2021-09-06 00:00:00,1630886400,1630886400


In [9]:
get_stock_concept_components('BK0708')

,close,PCT_CHG,delta,code,name,concept_symbol,concept_name,source,date,revision,date_stamp
concept_symbol,,,,,,,,,,,
BK0708,2.27,-0.0173,-0.04,600146,*ST环球,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0708,2.53,-0.0078,-0.02,600139,ST西源,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0708,2.96,-0.0166,-0.05,603555,ST贵人,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0708,19.95,0.0091,0.18,002780,三夫户外,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0708,5.90,0.0120,0.07,600018,上港集团,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
...,...,...,...,...,...,...,...,...,...,...,...
BK0708,70.62,0.0404,2.74,300999,金龙鱼,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0708,3.05,0.0304,0.09,601718,际华集团,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0708,9.87,-0.0199,-0.20,300162,雷曼光电,BK0708,体育产业,eastmoney,2021-09-06 00:00:00,1630886400,1630886400


In [10]:
ret_stock_concepts = get_stock_concepts(['002617', '601778'])

ret_stock_concepts.groupby(level=[0]).agg({'name': 'last', 
                                           u'close':'count' }).rename(columns={'close':u'股票数', 
                                                                               'name':u'概念版块'}).sort_values(by=[u'股票数',],
                                                                                                                ascending=[False,])

,概念版块,股票数
concept_symbol,,
BK0588,晶科科技,2
BK0867,晶科科技,2
BK0428,晶科科技,1
BK0815,晶科科技,1
BK0989,晶科科技,1
BK0978,晶科科技,1
BK0977,露笑科技,1
BK0952,露笑科技,1
BK0924,露笑科技,1


In [11]:
ret_stock_concepts

,close,PCT_CHG,delta,code,name,concept_symbol,concept_name,source,date,revision,date_stamp
concept_symbol,,,,,,,,,,,
BK0596,13.57,0.0319,0.42,002617,露笑科技,BK0596,融资融券,eastmoney,2021-08-31 00:00:00,1630368000,1630368000
BK0867,13.57,0.0319,0.42,002617,露笑科技,BK0867,富时罗素,eastmoney,2021-08-31 00:00:00,1630368000,1630368000
BK0804,13.06,0.0008,0.01,002617,露笑科技,BK0804,深股通,eastmoney,2021-09-01 00:00:00,1630454400,1630454400
BK0674,12.30,0.0000,0.00,002617,露笑科技,BK0674,蓝宝石,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK1006,12.30,0.0000,0.00,002617,露笑科技,BK1006,碳基材料,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0900,12.30,0.0000,0.00,002617,露笑科技,BK0900,新能源车,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0977,12.30,0.0000,0.00,002617,露笑科技,BK0977,碳化硅,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0574,12.30,0.0000,0.00,002617,露笑科技,BK0574,锂电池,eastmoney,2021-09-06 00:00:00,1630886400,1630886400
BK0588,12.30,0.0000,0.00,002617,露笑科技,BK0588,太阳能,eastmoney,2021-09-06 00:00:00,1630886400,1630886400


In [12]:
get_stock_concept_kline(['BK0588', 'BK0900'])

,,datetime,open,close,high,low,volume,amount,turnover,symbol,date,date_stamp
date,symbol,,,,,,,,,,,
2014-11-03,BK0588,2014-11-03,1122.51,1129.25,1129.33,1120.72,8936106.0,9.705709e+09,0.0077,BK0588,2014-11-03,1414972800
2014-11-04,BK0588,2014-11-04,1129.49,1130.75,1133.21,1122.93,9558812.0,1.011644e+10,0.0091,BK0588,2014-11-04,1415059200
2014-11-05,BK0588,2014-11-05,1132.83,1127.79,1134.83,1126.92,9631097.0,9.933424e+09,0.0070,BK0588,2014-11-05,1415145600
2014-11-06,BK0588,2014-11-06,1127.65,1133.90,1133.99,1120.86,8671601.0,9.821922e+09,0.0116,BK0588,2014-11-06,1415232000
2014-11-07,BK0588,2014-11-07,1137.08,1125.13,1142.07,1117.04,11148782.0,1.251673e+10,0.0221,BK0588,2014-11-07,1415318400
...,...,...,...,...,...,...,...,...,...,...,...,...
2021-08-30,BK0900,2021-08-30,1422.52,1430.35,1439.49,1420.34,133280399.0,2.712820e+11,0.0134,BK0900,2021-08-30,1630281600
2021-08-31,BK0900,2021-08-31,1428.68,1421.84,1428.68,1408.02,124555018.0,2.265599e+11,0.0144,BK0900,2021-08-31,1630368000
2021-09-01,BK0900,2021-09-01,1422.19,1390.45,1422.73,1373.91,140455468.0,2.485636e+11,0.0343,BK0900,2021-09-01,1630454400


In [13]:
def get_peak_high_price(logVs):
    logHs = [logVs[0]]
    for logV in logVs[1:]:
        if logV>logHs[-1]:
            logHs.append(logV)
        else:
            logHs.append(logHs[-1])
    logHs = np.array(logHs)
    return logHs


def getPoolRate(logVs):
    """
    # 对数积水率
    """
    logHs = get_peak_high_price(logVs)
    gain = np.sum(logVs) - logVs[0] * len(logVs)
    lost_and_gain = np.sum(logHs) - logHs[0] * len(logHs)
    ninfp1_to_n1p1 = lambda x: 2 / (2 - x) - 1
    poolRate = None
    if gain == 0:
        poolRate = 0
    elif lost_and_gain == 0:
        poolRate = float('-inf')
    else:
        poolRate = gain / lost_and_gain
    poolRate = ninfp1_to_n1p1(poolRate)
    return poolRate


def getPoolRate_vX(logVs,
                   regression,):
    """
    # 稳健对数积水率
    """
    logHs = get_peak_high_price(logVs)
    gain = np.sum(logVs) - logVs[0] * len(logVs)
    lost_and_gain = np.sum(logHs) - logHs[0] * len(logHs)

    lost = lost_and_gain - gain

    dts = np.arange(len(logVs))
    #s, m = get_coef_cut(dts, logVs)
    #predLogVs = s * dts + m
    #gain = (predLogVs[-1] - predLogVs[0]) * len(logVs) / 2
    gain = (regression[-1] - regression[0]) * len(logVs) / 2
    lost_and_gain = np.abs(lost + gain)
    ninfp1_to_n1p1 = lambda x: 2 / (2 - x) - 1

    poolRate = None
    if gain == 0:
        poolRate = 0
    elif lost_and_gain == 0:
        poolRate = float('-inf')
    else:
        poolRate = gain / lost_and_gain

    poolRate = ninfp1_to_n1p1(poolRate)
    return poolRate

ret_concept_kline = get_stock_concept_kline(['BK0733'])

In [14]:
ret_concept_kline = get_stock_concept_kline(['BK0437'])
print(ret_concept_kline)
ret_concept_np = ret_concept_kline[[AKA.HIGH]].values

from GolemQ.fractal.v0 import (
    ma30_cross_func,
    renko_trend_cross_func,
    zen_trend_func,
)
features = ma30_cross_func(ret_concept_kline)
features = renko_trend_cross_func(ret_concept_kline, features)
features = zen_trend_func(ret_concept_kline, features)
from GolemQ.fractal.v8 import (
    calc_zen_chopsticks,
)
features = calc_zen_chopsticks(ret_concept_kline,
                                             features,
                                             detect_buy3=False)   
        
ret_fractal_length = int(np.maximum(features[FLD.DEA_ZERO_TIMING_LAG],
                                    np.maximum(features[FLD.MA90_CLEARANCE_TIMING_LAG],
                                    np.maximum(features[FLD.ZEN_PEAK_TIMING_LAG],
                                               features[FLD.MA90_TREND_TIMING_LAG])))[-1])

print(ret_fractal_length)

ret_inter = getPoolRate(ret_concept_np[-ret_fractal_length:])
ret_inter_vX = getPoolRate_vX(ret_concept_np[-ret_fractal_length:],
                              features[FLD.LINEAREG_PRICE].values[-ret_fractal_length:])

                     datetime     open    close     high      low      volume  \
date       symbol                                                               
2014-11-03 BK0437  2014-11-03  4311.87  4317.96  4354.49  4289.04   9805388.0   
2014-11-04 BK0437  2014-11-04  4301.84  4403.50  4417.18  4280.30  14413011.0   
2014-11-05 BK0437  2014-11-05  4389.78  4315.32  4392.02  4311.90   9700095.0   
2014-11-06 BK0437  2014-11-06  4315.82  4302.73  4327.77  4254.29   6882909.0   
2014-11-07 BK0437  2014-11-07  4291.43  4242.95  4341.45  4227.76   7985498.0   
...                       ...      ...      ...      ...      ...         ...   
2021-08-30 BK0437  2021-08-30  8668.86  8916.61  8937.81  8618.32  26849222.0   
2021-08-31 BK0437  2021-08-31  8874.30  9326.00  9365.44  8790.45  30197733.0   
2021-09-01 BK0437  2021-09-01  9402.73  8959.36  9562.99  8892.17  32235273.0   
2021-09-02 BK0437  2021-09-02  8934.75  9488.69  9513.56  8934.75  28471125.0   
2021-09-03 BK0437  2021-09-0

In [15]:
ret_inter

array([0.75461999])

In [16]:
ret_inter_vX

array([0.82500847])

In [17]:
from GolemQ.fractal.v0 import (
        ma30_cross_func,
        renko_trend_cross_func,
        zen_trend_func,
    )        
from GolemQ.fractal.v8 import (
        calc_zen_chopsticks,
    )

concept_list = get_stock_concept_list()

print('BK0690' in concept_list['symbol'].to_list())

baned_concept_list = list(set(['BK0596', 'BK0571', 'BK0552', 'BK0815', 
                                           'BK0570', 'BK0683', 'BK0817', 'BK0528',
                                           'BK0879', 'BK0835', 'BK0571', 'BK0570', 
                                           'BK0816', 'BK0815', 'BK0980', 'BK0596',
                                            'BK0520', 'BK0571', 'BK0835', 'BK0803', 
                                            'BK0980', 'BK0815', 'BK0816']))
concept_list = concept_list.drop(index=concept_list.index.intersection(baned_concept_list))


super_concepts = []
super_concept_components = []

for symbol, concept in concept_list.iterrows():
    if (symbol == 'BK0690'):
        print(symbol, concept.at[AKA.NAME], concept)
    ret_concept_kline = get_stock_concept_kline(concept.name)

    try:
        ret_concept_np = ret_concept_kline[[AKA.HIGH]].values
        features = ma30_cross_func(ret_concept_kline)
        features = renko_trend_cross_func(ret_concept_kline, features)
        features = zen_trend_func(ret_concept_kline, features)
    except:
        print(symbol, concept[AKA.NAME], 'Failed', len(ret_concept_kline))
        continue
    features = calc_zen_chopsticks(ret_concept_kline,
                                                 features,
                                                 detect_buy3=False)   

    ret_fractal_length = int(np.maximum(features[FLD.DEA_ZERO_TIMING_LAG],
                                        np.maximum(features[FLD.MA90_CLEARANCE_TIMING_LAG],
                                        np.maximum(features[FLD.ZEN_PEAK_TIMING_LAG],
                                                   features[FLD.MA90_TREND_TIMING_LAG])))[-1])
    ret_fractal_length_df = pd.DataFrame([[ret_concept_np[-ret_fractal_length], 'overall']],
                                         columns=[AKA.HIGH, 'column'])
    
    all_columns = [FLD.DEA_ZERO_TIMING_LAG, FLD.MA90_CLEARANCE_TIMING_LAG, 
                   FLD.ZEN_PEAK_TIMING_LAG, FLD.MA90_TREND_TIMING_LAG]  # , FLD.MAPOWER30_PEAK_LOW_BEFORE
    for the_column in all_columns:
        if (features[the_column].values[-1] > 0) & \
           (features[the_column].values[-1] < len(features)):
            ret_fractal_length_df = ret_fractal_length_df.append({AKA.HIGH: ret_concept_np[-int(features[the_column].values[-1])], 
                                      'column': the_column}, ignore_index=True)
            try:
                if (ret_concept_np[-int(features[the_column].values[-1])] < ret_concept_np[-ret_fractal_length]):
                    #print(symbol, 'change {} to {}'.format(ret_concept_np[-ret_fractal_length], 
                    #                                   ret_concept_np[-int(features[the_column].values[-1])]))
                    ret_fractal_length = int(features[the_column].values[-1])
            except:
                #print(symbol, concept[AKA.NAME], 'Failed', features[the_column].values[-1], the_column, len(ret_concept_kline))
                continue
    
    if (len(features) < ret_fractal_length):
        #print(symbol, concept[AKA.NAME], 'Failed', len(ret_concept_kline))
        continue
        
    #print(symbol, ret_fractal_length_df)
    #print(symbol, ret_fractal_length, len(features))
    ret_inter = getPoolRate(ret_concept_np[-ret_fractal_length:])
    ret_inter_vX = getPoolRate_vX(ret_concept_np[-ret_fractal_length:],
                                  features[FLD.LINEAREG_PRICE].values[-ret_fractal_length:])
    if (features[FLD.ZEN_PEAK_TIMING_LAG].values[-1] > 0):
        ret_inter_vZ = getPoolRate_vX(ret_concept_np[-features[FLD.ZEN_PEAK_TIMING_LAG].values[-1]:],
                                  features[FLD.LINEAREG_PRICE].values[-features[FLD.ZEN_PEAK_TIMING_LAG].values[-1]:])
    else:
        ret_inter_vZ = np.array([0.0])
    
    if (((ret_inter + ret_inter_vX) > 1.68) or \
        ((ret_inter_vX + ret_inter_vZ * 2) > 2.472)) and \
        ((features[FLD.ZEN_BOOST_TIMING_LAG].values[-1] > 0) or \
        (features[FLD.ZEN_PEAK_TIMING_LAG].values[-1] > 0)):
        super_concepts.append((symbol, concept[AKA.NAME], ret_fractal_length, 
              features[FLD.ZEN_PEAK_TIMING_LAG].values[-1], 
              round(ret_inter[0], 4), round(ret_inter_vX[0], 4), round(ret_inter_vZ[0], 4),
              features[FLD.ZEN_BOOST_TIMING_LAG].values[-1],
              features[FLD.ZEN_DASH_TIMING_LAG].values[-1],
              features[FLD.ZEN_PEAK_TIMING_LAG].values[-1]))
        #print(symbol, concept[AKA.NAME], ret_fractal_length, 
        #      features[FLD.ZEN_PEAK_TIMING_LAG].values[-1], 
        #      round(ret_inter[0], 4) , round(ret_inter_vX, 4),  round(ret_inter_vZ, 4))
        ret_hot_concept_components = get_stock_concept_components(symbol)
        ret_hot_concept_components[FLD.POOL_COEF_RATIO] = [(round(ret_inter[0], 4), round(ret_inter_vX[0], 4), round(ret_inter_vZ[0], 4)) for fooo in ret_hot_concept_components.iterrows() ]
        super_concept_components.append(ret_hot_concept_components)
    
    if (symbol == 'BK0690'):
            print((symbol, concept[AKA.NAME], ret_fractal_length, 
              features[FLD.ZEN_PEAK_TIMING_LAG].values[-1], 
              round(ret_inter[0], 4), round(ret_inter_vX[0], 4), round(ret_inter_vZ[0], 4)),
              features[FLD.ZEN_BOOST_TIMING_LAG].values[-1],
              features[FLD.ZEN_DASH_TIMING_LAG].values[-1],
              features[FLD.ZEN_PEAK_TIMING_LAG].values[-1])
        
print(len(super_concepts), super_concepts)
ret_super_concept_components = pd.concat(super_concept_components, )

True
BK1008 国资云概念 Failed 2
BK1006 碳基材料 Failed 7
BK0989 储能 Failed 71
BK1004 工业母机 Failed 10
BK0690 氟化工 close                     4410.29
PCT_CHG                    0.0005
delta                        2.41
symbol                     BK0690
name                          氟化工
source                  eastmoney
date          2021-09-06 00:00:00
revision               1630886400
date_stamp             1630886400
Name: BK0690, dtype: object
('BK0690', '氟化工', 305, 140, 0.8277, 0.9165, 0.9308) -3 -5 140
BK0992 REITs概念 Failed 69
BK0998 机器视觉 Failed 45
BK1001 内贸流通 Failed 23
BK0993 宠物经济 Failed 60
BK0994 空间站概念 Failed 55
BK1003 抽水蓄能 Failed 12
BK0991 工程机械 Failed 70
BK0997 NFT概念 Failed 45
BK1005 专精特新 Failed 8
BK0995 华为昇腾 Failed 48
BK1002 激光雷达 Failed 15
BK1007 植物照明 Failed 4
BK0988 钠离子电池 Failed 73
BK0986 CAR-T细胞疗法 Failed 80
BK0987 盐湖提锂 Failed 74
BK0996 毛发医疗 Failed 47


C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\_data.py:179: RuntimeWarning: Mean of empty slice
  mean_ = np.nanmean(X, axis)
C:\ProgramData\Miniconda3\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\_data.py:187: RuntimeWarning: Mean of empty slice
  mean_1 = np.nanmean(Xr, axis=0)
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\_data.py:194: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn("Numerical issues were encountered "
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\preprocessing\_data.py:204: RuntimeWarning: Mean of empty slice
  mean_2 = np.nanmean(Xr, axis=0)
C:\ProgramData\Miniconda3\lib\site-packages\sklearn\prepro

BK0999 茅指数 Failed 31
BK1000 宁组合 Failed 24
BK1009 元宇宙概念 Failed 2
69 [('BK0478', '有色金属', 206, 56, 0.7159, 0.8028, 0.9041, 12, -3, 56), ('BK0537', '材料行业', 329, 140, 0.7824, 0.835, 0.864, -1, -5, 140), ('BK0731', '化肥行业', 289, -128, 0.8013, 0.8848, 0.0, 2, -3, -128), ('BK0424', '水泥建材', 27, 27, 0.9366, 0.9021, 0.9021, 4, 5, 27), ('BK0546', '玻璃陶瓷', 326, -135, 0.89, 0.8951, 0.0, 3, -4, -135), ('BK0545', '机械行业', 140, 140, 0.8947, 0.9052, 0.9052, 3, -5, 140), ('BK0910', '专用设备', 140, 140, 0.852, 0.8715, 0.8715, -1, 2, 140), ('BK0457', '输配电气', 84, 84, 0.9369, 0.9404, 0.9404, -2, -5, 84), ('BK0458', '仪器仪表', 87, 87, 0.8901, 0.8603, 0.8603, -2, 12, 87), ('BK0908', 'HIT电池', 122, 122, 0.9048, 0.9413, 0.9413, -3, -3, 122), ('BK0978', '光伏建筑一体化', 86, 86, 0.875, 0.8765, 0.8765, 3, -7, 86), ('BK0822', '租售同权', 2, -38, 1.0, 1.0, 0.0, 3, 11, -38), ('BK0918', '特高压', 83, 83, 0.9341, 0.9438, 0.9438, -2, 3, 83), ('BK0709', '赛马概念', 4, -60, 1.0, 1.0, 0.0, 4, -15, -60), ('BK0588', '太阳能', 425, -104, 0.8295, 0.871, 0.0

In [18]:
print(len(ret_super_concept_components[AKA.CODE].unique()), 
      len(ret_super_concept_components))

#super_concept_cache_file = 
import os
from GolemQ.utils.path import (
    mkdirs,
    mkdirs_user,
    cache_path,
    save_snapshot_cache,
    load_snapshot_cache,
)

save_snapshot_cache(os.path.join(mkdirs(cache_path('stock_cn', 
                                                        portable=True)), 
                                      'concepts'),
                            'super_concept.pickle', ret_super_concept_components)

super_concept_components = load_snapshot_cache(os.path.join(mkdirs(cache_path('stock_cn', 
                                                        portable=True)), 
                                      'concepts'),
                            'super_concept.pickle')
print(super_concept_components)
print('002815' in super_concept_components[AKA.CODE].to_list())
print(super_concept_components.query('code==\'002815\''))

3045 7370
                close  PCT_CHG  delta    code   name concept_symbol  \
concept_symbol                                                        
BK0478           3.83   0.0026   0.01  600595  *ST中孚         BK0478   
BK0478           2.31  -0.0043  -0.01  002501  *ST利源         BK0478   
BK0478          15.01  -0.0020  -0.03  601020   ST华钰         BK0478   
BK0478          15.40   0.0145   0.22  000962   东方钽业         BK0478   
BK0478           9.90  -0.0323  -0.33  002167   东方锆业         BK0478   
...               ...      ...    ...     ...    ...            ...   
BK0925          19.08   0.0016   0.03  603566    普莱柯         BK0925   
BK0925           8.88   0.0803   0.66  000507    珠海港         BK0925   
BK0925           7.80   0.0000   0.00  002797   第一创业         BK0925   
BK0925           6.68   0.1005   0.61  002349   精华制药         BK0925   
BK0925          10.73   0.1119   1.08  300011   鼎汉技术         BK0925   

               concept_name     source                 date    rev

In [19]:
from GolemQ.fetch.kline import (
    get_kline_price,
    get_kline_price_v2,
    get_kline_price_min,
)
start = '{}'.format(dt.now() - timedelta(days=1680))
data_day, stock_name_faked = get_kline_price('300821', start=start, verbose=True)
from GolemQ.fractal.v0 import (
    ma30_cross_func,
    renko_trend_cross_func,
    zen_trend_func,
)
features = ma30_cross_func(data_day.data)
features = renko_trend_cross_func(data_day.data, features)
features = zen_trend_func(data_day.data, features)
features = calc_zen_chopsticks(data_day.data,
                               features,
                               detect_buy3=False)
        
ret_fractal_length = int(np.maximum(features[FLD.DEA_ZERO_TIMING_LAG],
                                    np.maximum(features[FLD.MA90_CLEARANCE_TIMING_LAG],
                                    np.maximum(features[FLD.ZEN_PEAK_TIMING_LAG],
                                               features[FLD.MA90_TREND_TIMING_LAG])))[-1])
print(ret_fractal_length)
ret_inter = getPoolRate(data_day.data[AKA.HIGH].values[-ret_fractal_length:])
ret_inter_vX = getPoolRate_vX(data_day.data[AKA.HIGH].values[-ret_fractal_length:],
                              features[FLD.LINEAREG_PRICE].values[-ret_fractal_length:])

21-09-06 15:23 开始读取A股日K线历史数据 300821
时间戳差距超过： 3 days, 15:23:03.291358 尝试查找日线实盘数据.... ['300821']
追加实时实盘数据 1，股票代码：300821 时间：2021-09-06 00:00:00 价格：26.17
Code:300821, last time:2021-09-06 00:00:00 total bars:365
21-09-06 15:23 读取A股日K线历史数据完毕 300821 查询 A股 名称
21-09-06 15:23 读取A股日K线历史数据完毕 300821
Phase ma30_cross_func 2021-09-06 15:23:03
Phase renko_trend_cross_func 2021-09-06 15:23:03
slice: {0, 365} total:365
Phase renko_trend_cross_func Done 2021-09-06 15:23:03
Phase zen_trend_func 2021-09-06 15:23:03
2021-09-06 15:23:03 "300821" 自动划分为：10 段形态走势
Phase macd_cross_func 2021-09-06 15:23:03
Phase boll_cross_v2_func 2021-09-06 15:23:03
Phase boll_cross_func 2021-09-06 15:23:03
Phase bias_cross_func 2021-09-06 15:23:03
Phase bias_cross_func Phase 1 2021-09-06 15:23:03
Phase bias_cross_func Phase 2 2021-09-06 15:23:03
Phase bias_cross_func Done 2021-09-06 15:23:03
Phase maxfactor_cross_v2_func 2021-09-06 15:23:03
Phase lineareg_cross_v2_func 2021-09-06 15:23:03
Phase lineareg_cross_v2_func 2021-09-0

In [20]:
ret_inter

0.7944072732026608

In [21]:
ret_inter_vX

0.9029804076606613

In [22]:
features[FLD.ZEN_PEAK_TIMING_LAG]

date        code  
2020-03-12  300821    -1
2020-03-13  300821    -2
2020-03-16  300821    -3
2020-03-17  300821    -4
2020-03-18  300821    -5
                      ..
2021-08-31  300821    84
2021-09-01  300821    85
2021-09-02  300821    86
2021-09-03  300821    87
2021-09-06  300821    88
Name: ZEN_PEAK_LAG, Length: 365, dtype: int32